In [1]:
! pip install -q ollama openai python-dotenv instructor jsonschema

# Import ollama and verify it works


In [12]:
# load OPENAI_API_KEY from environment variables
import os
from dotenv import load_dotenv


load_dotenv()

# load OPENAI_API_KEY from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

print(openai_api_key[:5] + "..." + openai_api_key[-5:])


KEY_H..._HERE


In [67]:
from openai import OpenAI
import instructor

# clients = {
#   'openai': instructor.from_openai(OpenAI(api_key=openai_api_key)),
#   'ollama': instructor.from_openai(OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'))
# }

clients = {
  'openai': OpenAI(api_key=openai_api_key),
  'ollama': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
}

# models = ['aya', 'eurollm', 'gpt-4o-mini', 'llama3.1']
# models = ['aya', 'llama3.1', 'gpt-4o-mini', 'gpt-4o']
models = ['llama3.1', 'gpt-4o-mini', 'gpt-4o']
# models = ['gpt-4o-mini', 'gpt-4o']


model2client = {
  'aya': 'ollama',
  'llama3.1': 'ollama',
  'gpt-4o-mini': 'openai',
  'gpt-4o': 'openai',
}

def get_client(model_name):
  return clients[model2client[model_name]]

In [ ]:
response = get_client('llama3.1').completions.create(
  model='llama3.1',
  # messages=[
  #   {
  #     "role": "user",
  #     "content": "What is your name?",
  #   },
  # ],
  prompt="What is your name?",
  max_tokens=100,
  # response_model=str
)
response

Completion(id='cmpl-640', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='I don\'t have a personal name. I\'m an AI designed to provide information and assist with tasks, so I\'m often referred to as "Assistant" or "AI." However, if you\'d like, I can be addressed as "Assistant," "Help," or even just "Chat" for simplicity\'s sake! What\'s on your mind? Do you have any questions or topics you\'d like to discuss?')], created=1732647701, model='llama3.1', object='text_completion', system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=84, prompt_tokens=15, total_tokens=99, completion_tokens_details=None, prompt_tokens_details=None))

In [5]:
test_data_folder = "./test/raw/"
true_data_folder = "./test/labeled_case/"

# Read test data
import os

test_data = []
for file in os.listdir(test_data_folder):
    with open(test_data_folder + file, 'r') as f:
        test_data.append(f.read())

# Read true data
true_data = []
for file in os.listdir(true_data_folder):
    with open(true_data_folder + file, 'r') as f:
        true_data.append(f.read())


In [6]:
print(true_data[0])

Справа № 685/5/24
Провадження № 2/685/77/24 

У Х В А Л А
(про залишення позовної заяви без руху)

       02 січня 2024 року                                                            
        Суддя Теофіпольського районного суду Хмельницької області Самойлович А.П., розглянувши матеріали цивільної справи за позовом [ОСОБА_1|female|v_rod] ( АДРЕСА_1 ) до [ОСОБА_2|female|v_rod] ( АДРЕСА_1 ), третя особа: Виконавчий комітет Теофіпольської селищної ради (30602, вул.Небесної Сотні, 19, смт Теофіполь, Хмельницького району, Хмельницької області) про позбавлення батьківських прав, 

                                                     В С Т А Н О В И В :

02 січня 2023 року адвокат Сторожук Ю.В. в інтересах позивачки [ОСОБА_1|female|v_rod] звернувся до районного суду з позовом до відповідачки [ОСОБА_2|female|v_rod], заявивши вимогу позбавити її батьківських прав відносно малолітнього сина [ОСОБА_3|male|v_rod], ІНФОРМАЦІЯ_1.
Позовні вимоги обґрунтовані тим, що відповідачка ухиляється від викон

In [7]:
# labeled data format [ОСОБА_N|male|v_naz]
true_labels = {}

# regex for extracting the labels
import re
label_regex = re.compile(r'\[(ОСОБА_\d)\|(male|female|unknown)\|(v_naz|v_rod|v_dav|v_zna|v_oru|v_mis|v_kly)\]')

for idx, data in enumerate(true_data):
    labels = label_regex.findall(data)
    # group by entity
    true_labels[idx] = {}

    for label in labels:
        entity = label[0]

        if entity not in true_labels[idx]:
            true_labels[idx][entity] = {'gender': label[1], 'cases': []}

        true_labels[idx][entity]['cases'].append(label[2])

true_labels[0]


{'ОСОБА_1': {'gender': 'female', 'cases': ['v_rod', 'v_rod', 'v_rod']},
 'ОСОБА_2': {'gender': 'female', 'cases': ['v_rod', 'v_rod', 'v_rod']},
 'ОСОБА_3': {'gender': 'male', 'cases': ['v_rod']},
 'ОСОБА_4': {'gender': 'female', 'cases': ['v_naz']},
 'ОСОБА_5': {'gender': 'female', 'cases': ['v_naz']}}

In [39]:
from pydantic import BaseModel
from typing import Dict

def predict(model, prompt, document):
    # response = get_client(model).completions.create(
    #     model=model,
    #     prompt=prompt + document + "\n```ПОЧАТОК\n",
    #     max_tokens=100
    # )

    response = get_client(model).chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": document}
        ]
        # response_model=Label
    )

    return response

In [50]:
labeling_instructions = '''
Ти професійний лінгвіст, який навченмй визначати граматичний рід та відмінок сутності в тексті.

Тобі буде наданий текст, в якому буде наявна сутність "ОСОБА_".
Твоє завдання — вгадати граматичний та відмінок цієї сутності, беручи до уваги:
1. контекст тексту;
2. взаємодії з іншими сутностями;
3. залежностях від інших слів та сутностей;
4. слова, що оточують сутність.
5. інші фактори, які вважаєш важливими.

Сутність може зустрічатись у тексті декілька разів. Відмінок має бути визначений для кожної згадки.

Варіанти граматичного роду: male, female, unknown.
Варіанти відмінка:
1. 'v_naz' для називного. Відповідає на питання "хто? що?"
2. 'v_rod' для родового. Відповідає на питання "кого? чого?"
3. 'v_dav' для давального. Відповідає на питання "кому? чому?"
4. 'v_zna' для знахідного. Відповідає на питання "кого? що?"
5. 'v_oru' для орудного. Відповідає на питання "ким? чим?"
6. 'v_mis' для місцевого. Відповідає на питання "На кому? На чому?"
7. 'v_kly' для кличного

Відповідь має бути у форматі JSON.
Ось приклад відповіді:
{"ОСОБА_1": {"gender": "male", "cases": ["v_naz", "v_rod"]},"ОСОБА_2": {"gender": "female","cases": ["v_rod", "v_rod"]}}


Якщо ти не впевнений, вкажи "unknown".
Дотримуйся прикладу та форматування відповіді. Нічого, окрім правильного JSON, одним рядком, без переносів.
'''

In [ ]:
import json
from jsonschema import validate

def parse_prediction(prediction):
    entity_shema = {
        "type": "object",
        "properties": {
            "gender": {
                "type": "string",
                "enum": ["male", "female", "unknown"]
            },
            "cases": {
                "type": "array",
                "items": {
                    "type": "string",
                    "enum": ["v_naz", "v_rod", "v_dav", "v_zna", "v_oru", "v_mis", "v_kly"]
                }
            }
        }
    }

    schema = {
        "type": "object",
        "properties": {
            "ОСОБА_\d+": entity_shema
        }
    }

    try:
        prediction_obj = json.loads(prediction)
        if validate(instance=prediction_obj, schema=schema) is None:
            return prediction_obj
        else:
            return None
    except:
        return None


def run_prediction(limit=None):
    y_pred = {}

    for model in models:
        y_pred[model] = {}

    for model in models:
        y_pred[model] = {}
        for idx, data in enumerate(test_data if limit is None else test_data[:limit]):
            entities = [(entity + ' (зустрічається ' + str(len(true_labels[idx][entity]['cases'])) + ' рази)')  for entity in true_labels[idx].keys()]
            
            data = f'''
                Сутності особи в цьому тексті:
                {entities}
                Текст:
                {data}
            '''
            response = predict(model, labeling_instructions, data)
        
            y_pred[model][idx] = parse_prediction(response.choices[0].message.content)
    
    return y_pred

In [108]:
y_pred = run_prediction(limit=5)

for model in models:
    for idx in y_pred[model]:
        print(f'Model: {model}, Document: {idx}')
        print('True:', true_labels[idx])
        print('Predicted:', y_pred[model][idx])

Entities: ОСОБА_1 (зустрічається 3 рази), ОСОБА_2 (зустрічається 3 рази), ОСОБА_3 (зустрічається 1 рази), ОСОБА_4 (зустрічається 1 рази), ОСОБА_5 (зустрічається 1 рази)
Entities: ОСОБА_1 (зустрічається 4 рази)
Entities: ОСОБА_1 (зустрічається 2 рази), ОСОБА_2 (зустрічається 1 рази), ОСОБА_3 (зустрічається 2 рази)
Entities: ОСОБА_1 (зустрічається 11 рази), ОСОБА_2 (зустрічається 3 рази)
Entities: ОСОБА_1 (зустрічається 2 рази), ОСОБА_2 (зустрічається 3 рази)
Entities: ОСОБА_1 (зустрічається 3 рази), ОСОБА_2 (зустрічається 3 рази), ОСОБА_3 (зустрічається 1 рази), ОСОБА_4 (зустрічається 1 рази), ОСОБА_5 (зустрічається 1 рази)
Entities: ОСОБА_1 (зустрічається 4 рази)
Entities: ОСОБА_1 (зустрічається 2 рази), ОСОБА_2 (зустрічається 1 рази), ОСОБА_3 (зустрічається 2 рази)
Entities: ОСОБА_1 (зустрічається 11 рази), ОСОБА_2 (зустрічається 3 рази)
Entities: ОСОБА_1 (зустрічається 2 рази), ОСОБА_2 (зустрічається 3 рази)
Entities: ОСОБА_1 (зустрічається 3 рази), ОСОБА_2 (зустрічається 3 рази), ОС

In [100]:
y_pred

{'llama3.1': {0: None},
 'gpt-4o-mini': {0: {'ОСОБА_1': {'gender': 'female',
    'cases': ['v_naz', 'v_rod', 'v_naz']},
   'ОСОБА_2': {'gender': 'female', 'cases': ['v_naz', 'v_zna']},
   'ОСОБА_3': {'gender': 'unknown', 'cases': ['v_rod']},
   'ОСОБА_4': {'gender': 'unknown', 'cases': ['v_naz']},
   'ОСОБА_5': {'gender': 'unknown', 'cases': ['v_naz']}}},
 'gpt-4o': {0: {'ОСОБА_1': {'gender': 'female',
    'cases': ['v_rod', 'v_naz', 'v_rod']},
   'ОСОБА_2': {'gender': 'female', 'cases': ['v_rod', 'v_kly', 'v_rod']},
   'ОСОБА_3': {'gender': 'male', 'cases': ['v_rod']},
   'ОСОБА_4': {'gender': 'unknown', 'cases': ['v_zna']},
   'ОСОБА_5': {'gender': 'unknown', 'cases': ['v_rod']}}}}

In [109]:
# calculate accuracy for gender and cases separately
# True: {'ОСОБА_1': {'gender': 'female', 'cases': ['v_rod', 'v_rod', 'v_rod']}, 'ОСОБА_2': {'gender': 'female', 'cases': ['v_rod', 'v_rod', 'v_rod']}, 'ОСОБА_3': {'gender': 'male', 'cases': ['v_rod']}, 'ОСОБА_4': {'gender': 'female', 'cases': ['v_naz']}, 'ОСОБА_5': {'gender': 'female', 'cases': ['v_naz']}}
# Predicted: {'ОСОБА_1': {'gender': 'unknown', 'cases': ['v_naz', 'v_dav']}, 'ОСОБА_2': {'gender': 'female', 'cases': ['v_rod', 'v_dav', 'v_naz', 'v_zna']}, 'ОСОБА_3': {'gender': 'male', 'cases': ['v_naz']}, 'ОСОБА_4': {'gender': 'unknown'}, 'ОСОБА_5': {'gender': 'unknown'}}


def calculate_gender_accuracy(true_labels, predicted_labels):
    correct = 0
    total = 0

    for entity in true_labels:
        if entity in predicted_labels:
            total += 1
            if true_labels[entity] == predicted_labels[entity]:
                correct += 1
            
    return correct / total

# If the count is different, the number of correct predictions is 0, total is the number of true labels
# If the count is the same, calculate the accuracy as the number of correct predictions / total number of entities
def calculate_cases_accuracy(true_labels, predicted_labels):
    correct = 0
    total = 0

    for entity in true_labels:
        if entity in predicted_labels:
            total += len(true_labels[entity]['cases'])
            for case in true_labels[entity]['cases']:
                if case in predicted_labels[entity]['cases']:
                    correct += 1
        else:
            total += len(true_labels[entity]['cases'])

    return correct / total


for model in models:
    print(f'Model: {model}')
    print

    accuracy = 0

    for idx in y_pred[model]:
        if y_pred[model][idx] is not None:
            accuracy += calculate_gender_accuracy(true_labels[idx], y_pred[model][idx])
    
    accuracy /= len(y_pred[model])

    print(f'gender accuracy: {accuracy:.2f}')

    accuracy = 0

    for idx in y_pred[model]:
        if y_pred[model][idx] is not None:
            accuracy += calculate_cases_accuracy(true_labels[idx], y_pred[model][idx])

    accuracy /= len(y_pred[model])

    print(f'cases accuracy: {accuracy:.2f}')


Model: llama3.1
gender accuracy: 0.00
cases accuracy: 0.02
Model: gpt-4o-mini
gender accuracy: 0.00
cases accuracy: 0.75
Model: gpt-4o
gender accuracy: 0.04
cases accuracy: 0.85


# Prompt to detect a gender of an entity


In [142]:
system_instructions = '''
Ти професійний лінгвіст, який навченмй визначати граматичний рід.
Тобі буде наданий текст, в якому буде наявна сутність "ОСОБА_".
Твоє завдання — вгадати граматичний рід цієї сутності, беручи до уваги:
1. контекст тексту;
2. взаємодії з іншими сутностями;
3. залежностях від інших слів та сутностей;
4. слова, що оточують сутність.
5. інші фактори, які вважаєш важливими.

Сутність може зустрічатись у тексті декілька разів. Граматичний рід має бути визначений для кожної згадки.

Варіанти граматичного роду: male, female, unknown.

Відповідь має бути у форматі JSON.
Ось приклад відповіді:
{"ОСОБА_1": "male", "ОСОБА_2": "unknown"}

Якщо ти не впевнений, вкажи "unknown".
Дотримуйся прикладу та форматування відповіді. Нічого, окрім правильного JSON, одним рядком, без переносів.
'''

def parse_prediction(prediction):
    entity_shema = {
        "type": "string",
        "enum": ["male", "female", "unknown"]
    }

    schema = {
        "type": "object",
        "properties": {
            "ОСОБА_\d+": entity_shema
        }
    }

    try:
        prediction_obj = json.loads(prediction)
        if validate(instance=prediction_obj, schema=schema) is None:
            return prediction_obj
        else:
            return None
    except:
        return None

def run_prediction(limit=None):
    y_pred = {}

    for model in models:
        y_pred[model] = {}

    for model in models:
        y_pred[model] = {}
        for idx, data in enumerate(test_data if limit is None else test_data[:limit]):
            entities = [(entity + ' (зустрічається ' + str(len(true_labels[idx][entity]['cases'])) + ' рази)')  for entity in true_labels[idx].keys()]
            
            data = f'''
                Сутності особи в цьому тексті:
                {entities}
                Текст:
                {data}
            '''
            response = predict(model, system_instructions, data)
        
            y_pred[model][idx] = parse_prediction(response.choices[0].message.content)
    
    return y_pred

y_pred = run_prediction(limit=10)

In [ ]:

# calculate accuracy

for model in models:
    print(f'Model: {model}')
    print

    correct = 0
    total = 0
    
    for idx in true_labels:
        for entity in true_labels[idx]:
            total += 1
            if y_pred[model][idx] is not None and entity in y_pred[model][idx] and y_pred[model][idx][entity] is not None:
                true_value = true_labels[idx][entity]['gender']
                predicted_value = y_pred[model][idx][entity]
                if true_value == predicted_value:
                    correct += 1

    print(f'accuracy: {correct / total:.2f}')
    print

Model: llama3.1
accuracy: 0.22
Model: gpt-4o-mini
accuracy: 0.65
Model: gpt-4o
accuracy: 0.65
